In [30]:
import ee
import pandas as pd

ee.Initialize()

# 관심 지역 및 기간 설정
aoi = ee.Geometry.Polygon(
    [[[
                            128.852806,
                            35.003003
                        ],
                        [
                            128.852806,
                            35.044895
                        ],
                        [
                            128.940868,
                            35.044895
                        ],
                        [
                            128.940868,
                            35.003003
                        ],
                        [
                            128.852806,
                            35.003003
                        ]]]
)

start_date = '2021-01-01'
end_date = '2023-10-31'

# Sentinel-2 ImageCollection 필터링
sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
            .filterBounds(aoi) \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))

# NDWI와 FAI 계산
def calculate_indices(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    lambda_nir = 832.8
    lambda_red = 664.6
    lambda_swir1 = 1613.7

    # Selecting vand.
    red = image.select('B4')   # Red vand
    nir = image.select('B8')   # NIR vand
    swir1 = image.select('B11') # SWIR1 vand
    # Calculate FAI
    fai = nir.subtract(red).add(
        swir1.subtract(red).multiply(
            (lambda_nir - lambda_red) / (lambda_swir1 - lambda_red)
        )
    ).rename('FAI')
    return image.addBands([ndwi, fai])

# 조건에 맞는 픽셀 수 계산
def count_pixels(image):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    masked = image.updateMask(image.select('NDWI').gt(0.5))
    fai_count = masked.select('FAI').gt(-5).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=10
    ).get('FAI')
    return ee.Feature(None, {'date': date, 'fai_count': fai_count})

# 이미지 컬렉션에 NDWI와 FAI 계산 적용 후 픽셀 수 계산
processed_collection = sentinel2.map(calculate_indices)
pixel_counts = processed_collection.map(count_pixels)

# 결과를 서버 측 객체로 변환 (Python 클라이언트로 가져오기)
pixel_count_features = pixel_counts.getInfo()['features']

# 결과를 pandas DataFrame으로 변환
df = pd.DataFrame([feat['properties'] for feat in pixel_count_features])

# DataFrame을 'date' 컬럼에 따라 오름차순으로 정렬
df = df.sort_values(by='date')


In [31]:
df.describe()

,fai_count
count,87.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


In [29]:
# Calculate the area of the AOI in square meters.
area = aoi.area().getInfo()

# Convert the area to square kilometers (1 square kilometer = 1,000,000 square meters).
area_sq_km = area / 1_000_000

print("Area of the AOI in square kilometers:", area_sq_km)

Area of the AOI in square kilometers: 37.353208256071845


In [36]:
import ee
import pandas as pd

ee.Initialize()

# 관심 지역 및 기간 설정
aoi = ee.Geometry.Polygon(
    [[[
        128.852806, 35.003003
    ], [
        128.852806, 35.044895
    ], [
        128.940868, 35.044895
    ], [
        128.940868, 35.003003
    ], [
        128.852806, 35.003003
    ]]]
)
start_date = '2021-01-01'
end_date = '2023-10-31'

# Sentinel-2 ImageCollection 필터링
sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
            .filterBounds(aoi) \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))

# NDWI와 FAI 계산
def calculate_indices(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    lambda_nir = 832.8
    lambda_red = 664.6
    lambda_swir1 = 1613.7

    red = image.select('B4')   # Red band
    nir = image.select('B8')   # NIR band
    swir1 = image.select('B11') # SWIR1 band

    # Calculate FAI
    fai = nir.subtract(red).add(
        swir1.subtract(red).multiply(
            (lambda_nir - lambda_red) / (lambda_swir1 - lambda_red)
        )
    ).rename('FAI')
    return image.addBands([ndwi, fai])

# NDWI가 0.5 이상이고 FAI가 50 이상인 지역의 면적 계산
def calculate_area(image):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    mask = image.select('NDWI').gt(0.1).And(image.select('FAI').gt(-50))
    area = mask.multiply(ee.Image.pixelArea()).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=10  # Sentinel-2 픽셀 해상도
    ).get('NDWI')
    return ee.Feature(None, {'date': date, 'area': area})

# 이미지 컬렉션에 NDWI와 FAI 계산 적용 후 면적 계산
processed_collection = sentinel2.map(calculate_indices)
area_series = processed_collection.map(calculate_area)

# 결과를 서버 측 객체로 변환 (Python 클라이언트로 가져오기)
area_features = area_series.getInfo()['features']

# 결과를 pandas DataFrame으로 변환
df = pd.DataFrame([feat['properties'] for feat in area_features])

# DataFrame을 'date' 컬럼에 따라 오름차순으로 정렬
df = df.sort_values(by='date')


In [37]:
df

,area,date
0,42638.362166,2021-01-04
1,282436.305985,2021-01-09
2,81865.357758,2021-01-14
3,232386.609548,2021-01-19
4,123798.758484,2021-01-29
...,...,...
82,56893.975802,2023-09-06
83,2602.076248,2023-10-01
84,21204.781332,2023-10-11
85,900.717812,2023-10-16


In [38]:
# Sentinel-2 ImageCollection 필터링
sentinel2_image = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
            .filterBounds(aoi) \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
            .first()

In [ ]:
# 처리된 이미지를 지도에 추가 (수중식물 지수 사용) - 수상만 표시
uvi_params = {
    'bands': ['WEVI'],  # UVI 밴드만 사용
    'min': -0.4, # 수중식물 지수의 최소값
    'max': 1,   # 수중식물 지수의 최대값
    # 'palette': ['purple', 'blue', 'green', 'yellow', 'red']  # 색상 팔레트 설정
    'palette': ['#ffffb2','#fecc5c','#fd8d3c','#f03b20','#bd0026']  # 색상 팔레트 설정
}